# Hyperparameter Tuning - Classificação de Spam

## Objetivo

Otimizar hiperparâmetros do melhor modelo selecionado no notebook 02 para melhorar a performance.

## Estratégia

Usar RandomizedSearchCV para testar combinações aleatórias de hiperparâmetros:
- Mais rápido que GridSearchCV
- Geralmente encontra bons resultados
- Testa espaço de busca maior

## Modelo Base

Utilizar **SVM (LinearSVC)** - melhor modelo identificado no notebook 02.


In [4]:
# Imports
import pandas as pd
import numpy as np
import joblib
import time
from pathlib import Path
from datetime import datetime

# Scikit-learn
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report
)

# Modelo e pré-processamento
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder

print("Bibliotecas carregadas")


Bibliotecas carregadas


## Passo 1: Carregar Dados

Carregar os dados processados do notebook 01.


In [5]:
# Carregar data splits
splits_path = Path('artifacts/data_splits.joblib')

if not splits_path.exists():
    raise FileNotFoundError(
        "Artefatos do notebook 01 não encontrados. "
        "Execute primeiro o notebook 01_exploratory_analysis.ipynb"
    )

data_splits = joblib.load(splits_path)

X_train_tfidf = data_splits['X_train_tfidf']
X_test_tfidf = data_splits['X_test_tfidf']
y_train = data_splits['y_train']
y_test = data_splits['y_test']

print("Dados carregados com sucesso!")
print(f"X_train_tfidf: {X_train_tfidf.shape}")
print(f"X_test_tfidf: {X_test_tfidf.shape}")

# Converter labels para numéricos
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Mapeamento para referência
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(f"\nMapeamento de labels: {label_mapping}")
print(f"Ham = {label_mapping.get('ham', 'N/A')}, Spam = {label_mapping.get('spam', 'N/A')}")


Dados carregados com sucesso!
X_train_tfidf: (66758, 5000)
X_test_tfidf: (16690, 5000)

Mapeamento de labels: {'ham': np.int64(0), 'spam': np.int64(1)}
Ham = 0, Spam = 1


## Passo 2: Configurar Modelo Base

Configurar SVM (LinearSVC) como modelo base para otimização.


In [ ]:
# Modelo base: SVM (LinearSVC) - melhor modelo identificado no notebook 02
best_model_name = "SVM (LinearSVC)"
base_model = LinearSVC(random_state=42, max_iter=2000, dual=False)

# Espaço de busca de hiperparâmetros para LinearSVC
param_distributions = {
    'C': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
    'penalty': ['l2'],  # l1 requer loss='squared_hinge' e dual=True, mas dual=False é mais rápido
    'loss': ['squared_hinge']  # Única loss compatível com dual=False
}

print(f"Modelo base: {best_model_name}")
print(f"Parâmetros a otimizar: {list(param_distributions.keys())}")
print(f"\nEspaço de busca:")
print(f"  C: {param_distributions['C']}")
print(f"  penalty: {param_distributions['penalty']}")
print(f"  loss: {param_distributions['loss']}")


Modelo base: SVM (LinearSVC)
Parâmetros a otimizar: ['C', 'penalty', 'loss']

Espaço de busca:
  C: [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
  penalty: ['l2']
  loss: ['squared_hinge']


In [7]:
# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_distributions,
    n_iter=20,  # Número de combinações a testar
    cv=5,  # 5-fold cross-validation
    scoring='f1',  # Métrica a otimizar
    n_jobs=-1,  # Usar todos os cores
    random_state=42,
    verbose=1
)

print("Iniciando otimização de hiperparâmetros...")
print(f"Testando {random_search.n_iter} combinações com 5-fold CV")
print("Isso pode levar alguns minutos...\n")

start_time = time.time()

# LinearSVC precisa de labels numéricos
if 'SVM' in best_model_name or 'LinearSVC' in best_model_name:
    y_train_model = y_train_encoded
else:
    y_train_model = y_train

# Executar busca
random_search.fit(X_train_tfidf, y_train_model)

elapsed_time = time.time() - start_time

print(f"\nOtimização concluída em {elapsed_time:.2f}s ({elapsed_time/60:.2f} minutos)")
print(f"\nMelhores parâmetros encontrados:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\nMelhor F1-Score (CV): {random_search.best_score_:.4f}")


Iniciando otimização de hiperparâmetros...
Testando 20 combinações com 5-fold CV
Isso pode levar alguns minutos...

Fitting 5 folds for each of 6 candidates, totalling 30 fits


/home/lucas-biason/Projetos/Projetos/ml-spam-classifier-api/notebooks/venv/lib/python3.13/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=20. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(



Otimização concluída em 11.71s (0.20 minutos)

Melhores parâmetros encontrados:
  penalty: l2
  loss: squared_hinge
  C: 0.5

Melhor F1-Score (CV): 0.9848


## Passo 4: Avaliar Modelo Otimizado

Comparar performance do modelo otimizado com o modelo original.


In [8]:
# Treinar modelo original (baseline)
baseline_model = base_model.__class__(**base_model.get_params())
if 'SVM' in best_model_name or 'LinearSVC' in best_model_name:
    baseline_model.fit(X_train_tfidf, y_train_encoded)
    y_pred_baseline = baseline_model.predict(X_test_tfidf)
    y_pred_baseline = label_encoder.inverse_transform(y_pred_baseline)
else:
    baseline_model.fit(X_train_tfidf, y_train)
    y_pred_baseline = baseline_model.predict(X_test_tfidf)

# Predições do modelo otimizado
y_pred_optimized = random_search.best_estimator_.predict(X_test_tfidf)
if 'SVM' in best_model_name or 'LinearSVC' in best_model_name:
    y_pred_optimized = label_encoder.inverse_transform(y_pred_optimized)

# Métricas baseline
baseline_accuracy = accuracy_score(y_test, y_pred_baseline)
baseline_precision = precision_score(y_test, y_pred_baseline, pos_label='spam', zero_division=0)
baseline_recall = recall_score(y_test, y_pred_baseline, pos_label='spam', zero_division=0)
baseline_f1 = f1_score(y_test, y_pred_baseline, pos_label='spam', zero_division=0)

# Métricas otimizado
optimized_accuracy = accuracy_score(y_test, y_pred_optimized)
optimized_precision = precision_score(y_test, y_pred_optimized, pos_label='spam', zero_division=0)
optimized_recall = recall_score(y_test, y_pred_optimized, pos_label='spam', zero_division=0)
optimized_f1 = f1_score(y_test, y_pred_optimized, pos_label='spam', zero_division=0)

# Comparação
print("COMPARAÇÃO: Baseline vs Otimizado\n")
print("=" * 80)
print(f"{'Métrica':<20} {'Baseline':<15} {'Otimizado':<15} {'Melhoria':<15}")
print("=" * 80)
print(f"{'Accuracy':<20} {baseline_accuracy:<15.4f} {optimized_accuracy:<15.4f} {optimized_accuracy - baseline_accuracy:+.4f}")
print(f"{'Precision':<20} {baseline_precision:<15.4f} {optimized_precision:<15.4f} {optimized_precision - baseline_precision:+.4f}")
print(f"{'Recall':<20} {baseline_recall:<15.4f} {optimized_recall:<15.4f} {optimized_recall - baseline_recall:+.4f}")
print(f"{'F1-Score':<20} {baseline_f1:<15.4f} {optimized_f1:<15.4f} {optimized_f1 - baseline_f1:+.4f}")
print("=" * 80)

# Melhoria percentual
improvement = ((optimized_f1 - baseline_f1) / baseline_f1) * 100 if baseline_f1 > 0 else 0
print(f"\nMelhoria no F1-Score: {improvement:+.2f}%")


COMPARAÇÃO: Baseline vs Otimizado

Métrica              Baseline        Otimizado       Melhoria       
Accuracy             0.9857          0.9865          +0.0007
Precision            0.9830          0.9832          +0.0001
Recall               0.9900          0.9912          +0.0013
F1-Score             0.9865          0.9872          +0.0007

Melhoria no F1-Score: +0.07%


## Passo 5: Salvar Modelo Otimizado

Salvar o melhor modelo encontrado para uso no próximo notebook.


In [11]:
# Criar diretório para artefatos
artifacts_dir = Path('artifacts')
artifacts_dir.mkdir(exist_ok=True)

# Salvar modelo otimizado
optimized_model_path = artifacts_dir / 'optimized_model.joblib'
joblib.dump(random_search.best_estimator_, optimized_model_path)

# Salvar resultados da otimização
optimization_results = {
    'best_params': random_search.best_params_,
    'best_cv_score': random_search.best_score_,
    'test_accuracy': optimized_accuracy,
    'test_precision': optimized_precision,
    'test_recall': optimized_recall,
    'test_f1': optimized_f1,
    'baseline_f1': baseline_f1,
    'improvement': improvement,
    'model_type': best_model_name,
    'optimization_date': datetime.now().isoformat()
}

results_path = artifacts_dir / 'hyperparameter_tuning_results.joblib'
joblib.dump(optimization_results, results_path)

print("Artefatos salvos:")
print(f"- Modelo otimizado: {optimized_model_path}")
print(f"- Resultados: {results_path}")


Artefatos salvos:
- Modelo otimizado: artifacts/optimized_model.joblib
- Resultados: artifacts/hyperparameter_tuning_results.joblib


## Conclusão do Hyperparameter Tuning

### Resultados

O modelo foi otimizado e os resultados estão disponíveis acima.

### Próximo Passo

**Notebook 04: Pipeline Final**
- Treinar modelo final com todos os dados (sem split)
- Exportar modelo e vetorizador para produção
- Criar pipeline completo de inferência
